# Data analysis books for UBike services
We require to install **pandas**, **metplotlib** and **seaborn** in the virtual environment. 


In [58]:
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display

#pd.set_option('display.max_rows',30)
#pd.set_option('large_repr', 'info')

# read data here
bikecsv = pd.read_csv('data/youbike_240518.csv') 

bikecsv

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407030 entries, 0 to 407029
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   _id              407030 non-null  object
 1   station_no       407030 non-null  int64 
 2   station_name     407030 non-null  object
 3   station_name_en  407030 non-null  object
 4   district         407030 non-null  object
 5   district_en      407030 non-null  object
 6   total            407030 non-null  int64 
 7   rentAvailable    407030 non-null  int64 
 8   returnAvaiable   407030 non-null  int64 
 9   updated_time     407030 non-null  object
 10  info_time        407030 non-null  object
dtypes: int64(4), object(7)
memory usage: 34.2+ MB

## Before analyses, it requires us to know columns in csv data

**Here is the table show meaning or types of data.**

| Column Name | Type (Numerical/Category) | nullable | description |
| --- | --- | --- | --- |
| _id | Categorical | non-null | uuid generated by mongoDB also serial id |  
| station_no | Categorical | non-null | Serial Number |  
| station_name | Categorical | non-null | Station name |  
| district | Categorical | non-null | stations in which districts |  
| total | Numerical |  non-null | total = used + empty| 
| rentAvailable | Numerical |  non-null | available rental seats in stations | 
| returnAvaiable | Numerical |  non-null | avaiable return seats in stations | 
| updated_time | Date String | non-null | date time string in TZ GMT+8 |
|info_time | Date String | non-null | date time string in TZ GMT+8 |

Also, `info_time` can be categorize as workday or weekend to analyse 

## After Identify meanings of data columns, Here is a check list to process the dataframe

In [59]:
import ipywidgets as widgets

cb1 = widgets.Checkbox(description="define the outliers according to description of columns")
cb2 = widgets.Checkbox(description="define the error data in csv and remove them")
cb3 = widgets.Checkbox(description="process date string with pandas")
cb4 = widgets.Checkbox(description="Drop Some serial ID columns")

pb = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

def on_button_clicked(b):
    print("clicked on:",b)
    print("cb1:", cb1.value)
    print("cb2:", cb2.value)
    print("cb3:", cb3.value)
    print("cb4:", cb4.value)

pb.on_click(on_button_clicked)

ui = widgets.VBox([widgets.VBox([cb1, cb2, cb3, cb4]),pb])

display(ui)

clicked on: Button(description='Click me', style=ButtonStyle(), tooltip='Click me')
cb1: False
cb2: False
cb3: False
cb4: False


In [60]:
print('In the Step: we are going to drop serial columns')
# drop column here axis=0 indicates data in row, axis=1 indicates data in column
bikecsv = bikecsv.drop(axis=1, columns=['_id', 'station_no'])
bikecsv

In the Step: we are going to drop serial columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407030 entries, 0 to 407029
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   station_name     407030 non-null  object
 1   station_name_en  407030 non-null  object
 2   district         407030 non-null  object
 3   district_en      407030 non-null  object
 4   total            407030 non-null  int64 
 5   rentAvailable    407030 non-null  int64 
 6   returnAvaiable   407030 non-null  int64 
 7   updated_time     407030 non-null  object
 8   info_time        407030 non-null  object
dtypes: int64(3), object(6)
memory usage: 27.9+ MB

In [68]:
print('In the Step: we are going to drop error data')

filtered = bikecsv.loc[bikecsv['total'] != bikecsv['rentAvailable']+bikecsv['returnAvaiable'],:]

print('Error data Percentage: '+str(round(len(filtered)/len(bikecsv),4) * 100)+'%')

bikecsv = bikecsv.loc[bikecsv['total'] == bikecsv['rentAvailable']+bikecsv['returnAvaiable'],:]


In the Step: we are going to drop error data
Error data Percentage: 12.22%
